In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path = "/content/drive/MyDrive/genai/RAG/textclassif"
%cd {path}

/content/drive/MyDrive/genai/RAG/textclassif


In [4]:
#!pip install kagglehub==0.3.7

In [5]:
pwd

'/content/drive/MyDrive/genai/RAG/textclassif'

In [6]:
# Descargar Data
import kagglehub

path_data = "/content/drive/MyDrive/genai/RAG/textclassif/data"
!mkdir {path_data}
# Download latest version
path_data = kagglehub.dataset_download("asaniczka/product-titles-text-classification")

print("Path to dataset files:", path_data)

mkdir: cannot create directory ‘/content/drive/MyDrive/genai/RAG/textclassif/data’: File exists


100%|██████████| 194M/194M [00:01<00:00, 119MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/asaniczka/product-titles-text-classification/versions/3


In [7]:
!ls -l {path_data}

total 769952
-rw-r--r-- 1 root root 788424413 Feb  7 02:59 titles_to_categories.csv


In [8]:
# Leer data descargda de kaggle
import pandas as pd
data = pd.read_csv(path_data + "/titles_to_categories.csv")
data.head()

,title,category_name
0,"Green Leaf WW3D Wonder Extension Cord Winder, ...",Industrial Scientific
1,8pcs Toilet Seat Bumpers Universal Toilet Repl...,Industrial Scientific
2,YaeCCC 19 Pcs Hole Saw Kit 3/4''(19mm)- 6''(15...,Industrial Scientific
3,LLPT Butyl Putty Tape White 1 Inch x 33Ft Tigh...,Industrial Scientific
4,"Lightbeam 16"" Long Stem Deep Fry Thermometer w...",Industrial Scientific


In [9]:
data.shape

(5389901, 2)

In [12]:
data.groupby("category_name").count().sort_values(by="title", ascending=False)

,title
category_name,
Sports & Outdoors,720050
Baby,36154
Men's Shoes,32143
Women's Handbags,29934
Women's Shoes,28668
...,...
Garment Bags,39
Gift Cards,29
Smart Speakers,25


In [35]:
# Dividir en train y test
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.2, random_state=42, shuffle=True)

In [36]:
train.shape, test.shape

((4311920, 2), (1077981, 2))

In [15]:
!pip install langchain openai pandas faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 36.9 MB/s eta 0:00:00


In [18]:
# instalar dotenv
!pip install python-dotenv

In [19]:
# Configurar clave de openai con dotenv
import os
from dotenv import load_dotenv
load_dotenv()

True

In [27]:
test['title'].iloc[0]

"Arrow Shed Classic 8' x 6' Outdoor Padlockable Steel Storage Shed Building, Sage Green"

In [29]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00


In [32]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.7 MB/s eta 0:00:00


In [37]:
# Sacar una muestra aleaatoria de 10000 ejemplos de train
train_sample = train.sample(n=10000, random_state=42)

In [50]:
%%time
import pandas as pd
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.schema import Document

# Carga el dataset CSV
df = train_sample.copy() #pd.read_csv("tu_dataset.csv")

# Combina las columnas relevantes en un solo campo de contexto
df['context'] = df.apply(lambda row: f"Texto: {row['title']} | Entidad: {row['category_name']}", axis=1)

# Crear documentos para el índice
documents = [Document(page_content=row['context']) for _, row in df.iterrows()]

# Generar las embeddings utilizando OpenAI
embeddings = OpenAIEmbeddings()

# Crear el índice utilizando FAISS
vector_store = FAISS.from_documents(documents, embeddings)

# Inicializa el modelo de lenguaje
chat = ChatOpenAI(model="gpt-4o", temperature=0.01)

# Define el prompt para la clasificación
prompt_template = """
Eres un modelo encargado de clasificar texto en entidades específicas basándote en ejemplos previos.
Se te proporcionará un texto nuevo y ejemplos relevantes del dataset. Analiza los ejemplos y clasifica el texto.

Ejemplos relevantes:
{retrieved_examples}

Texto nuevo: {new_text}

Por favor, solo proporciona la entidad encontrada para este texto
"""

prompt = PromptTemplate(input_variables=["retrieved_examples", "new_text"], template=prompt_template)

def clasificar_texto(nuevo_texto, k):
    # Recuperar los ejemplos relevantes del índice
    docs_relevantes = vector_store.similarity_search(nuevo_texto, k=k)
    ejemplos_recuperados = "\n".join([doc.page_content for doc in docs_relevantes])

    # Crear el prompt para el modelo
    formatted_prompt = prompt.format(retrieved_examples=ejemplos_recuperados, new_text=nuevo_texto)

    # Clasificación utilizando el modelo
    response = chat.predict(formatted_prompt)
    return response.strip()



CPU times: user 11.4 s, sys: 800 ms, total: 12.2 s
Wall time: 31.7 s


In [51]:
# Ejemplo de uso variando el parámetro K
k=1
nuevo_texto = test['title'].iloc[0]
entidad_clasificada = clasificar_texto(nuevo_texto, k)
print(f"Texto de entrada:{nuevo_texto}")
print(f"Entidad clasificada: {entidad_clasificada}")
print(f"Texto real:{test['category_name'].iloc[0]}")

Texto de entrada:Arrow Shed Classic 8' x 6' Outdoor Padlockable Steel Storage Shed Building, Sage Green
Entidad clasificada: Home Storage & Organization
Texto real:Outdoor Storage  Housing


In [53]:
%%time
# Ejemplo de uso
k=2
nuevo_texto = test['title'].iloc[0]
entidad_clasificada = clasificar_texto(nuevo_texto, k)
print(f"Texto de entrada:{nuevo_texto}")
print(f"Entidad clasificada: {entidad_clasificada}")
print(f"Texto real:{test['category_name'].iloc[0]}")

Texto de entrada:Arrow Shed Classic 8' x 6' Outdoor Padlockable Steel Storage Shed Building, Sage Green
Entidad clasificada: Garden Storage & Housing
Texto real:Outdoor Storage  Housing
CPU times: user 43.7 ms, sys: 958 µs, total: 44.7 ms
Wall time: 821 ms


In [54]:
%%time
# Ejemplo de uso
k = 3
nuevo_texto = test['title'].iloc[0]
entidad_clasificada = clasificar_texto(nuevo_texto, k)
print(f"Texto de entrada:{nuevo_texto}")
print(f"Entidad clasificada: {entidad_clasificada}")
print(f"Texto real:{test['category_name'].iloc[0]}")

Texto de entrada:Arrow Shed Classic 8' x 6' Outdoor Padlockable Steel Storage Shed Building, Sage Green
Entidad clasificada: Outdoor Storage Housing
Texto real:Outdoor Storage  Housing
CPU times: user 50.2 ms, sys: 3.89 ms, total: 54.1 ms
Wall time: 839 ms


In [55]:
%%time
# Ejemplo de uso
k = 4
nuevo_texto = test['title'].iloc[0]
entidad_clasificada = clasificar_texto(nuevo_texto, k)
print(f"Texto de entrada:{nuevo_texto}")
print(f"Entidad clasificada: {entidad_clasificada}")
print(f"Texto real:{test['category_name'].iloc[0]}")

Texto de entrada:Arrow Shed Classic 8' x 6' Outdoor Padlockable Steel Storage Shed Building, Sage Green
Entidad clasificada: Outdoor Storage Housing
Texto real:Outdoor Storage  Housing
CPU times: user 52.7 ms, sys: 2.07 ms, total: 54.8 ms
Wall time: 911 ms


In [56]:
%%time
# Ejemplo de uso
k = 5
nuevo_texto = test['title'].iloc[0]
entidad_clasificada = clasificar_texto(nuevo_texto, k)
print(f"Texto de entrada:{nuevo_texto}")
print(f"Entidad clasificada: {entidad_clasificada}")
print(f"Texto real:{test['category_name'].iloc[0]}")

Texto de entrada:Arrow Shed Classic 8' x 6' Outdoor Padlockable Steel Storage Shed Building, Sage Green
Entidad clasificada: Outdoor Storage Housing
Texto real:Outdoor Storage  Housing
CPU times: user 41.1 ms, sys: 4.01 ms, total: 45.1 ms
Wall time: 907 ms


In [57]:
# Ejemplo de uso
k = 5
for index in range(10):
    print(f'Exmaple{index}:\n')
    nuevo_texto = test['title'].iloc[index]
    entidad_clasificada = clasificar_texto(nuevo_texto, k)
    print(f"Texto de entrada:{nuevo_texto}")
    print(f"Entidad clasificada: {entidad_clasificada}")
    print(f"Entidad real:{test['category_name'].iloc[index]}")
    print('\n')

Exmaple0:

Texto de entrada:Arrow Shed Classic 8' x 6' Outdoor Padlockable Steel Storage Shed Building, Sage Green
Entidad clasificada: Outdoor Storage Housing
Entidad real:Outdoor Storage  Housing


Exmaple1:

Texto de entrada:Berghaus Women's 24/7 Half Zip Long Sleeve Tech T-Shirt
Entidad clasificada: Sports & Outdoors
Entidad real:Sports & Outdoors


Exmaple2:

Texto de entrada:SilverAlloy 45% Ag BAg-5 18" Flux Coated Silver Solder Brazing Rods 3 Sticks
Entidad clasificada: Welding & Soldering
Entidad real:Welding & Soldering


Exmaple3:

Texto de entrada:Get Naked Bath Mat Cute Pink and White Bathroom Rugs Funny Non Slip Bathtub Decor Mats Super Absorbent Floor Carpet Machine Washable Bahtmat for Tub, Shower, Bedroom 16"x24"
Entidad clasificada: Bath Body
Entidad real:Home Décor Products


Exmaple4:

Texto de entrada:HP JC900A Tippingpoint 80mm Fan Module - Other Accessories
Entidad clasificada: Other Accessories
Entidad real:KVM Switches


Exmaple5:

Texto de entrada:Stampcolour P

In [58]:
# Ejemplo de uso
k = 10
for index in range(10):
    print(f'Exmaple{index}:\n')
    nuevo_texto = test['title'].iloc[index]
    entidad_clasificada = clasificar_texto(nuevo_texto, k)
    print(f"Texto de entrada:{nuevo_texto}")
    print(f"Entidad clasificada: {entidad_clasificada}")
    print(f"Entidad real:{test['category_name'].iloc[index]}")
    print('\n')

Exmaple0:

Texto de entrada:Arrow Shed Classic 8' x 6' Outdoor Padlockable Steel Storage Shed Building, Sage Green
Entidad clasificada: Outdoor Storage Housing
Entidad real:Outdoor Storage  Housing


Exmaple1:

Texto de entrada:Berghaus Women's 24/7 Half Zip Long Sleeve Tech T-Shirt
Entidad clasificada: Sports & Outdoors
Entidad real:Sports & Outdoors


Exmaple2:

Texto de entrada:SilverAlloy 45% Ag BAg-5 18" Flux Coated Silver Solder Brazing Rods 3 Sticks
Entidad clasificada: Welding & Soldering
Entidad real:Welding & Soldering


Exmaple3:

Texto de entrada:Get Naked Bath Mat Cute Pink and White Bathroom Rugs Funny Non Slip Bathtub Decor Mats Super Absorbent Floor Carpet Machine Washable Bahtmat for Tub, Shower, Bedroom 16"x24"
Entidad clasificada: Bath Products
Entidad real:Home Décor Products


Exmaple4:

Texto de entrada:HP JC900A Tippingpoint 80mm Fan Module - Other Accessories
Entidad clasificada: Other Accessories
Entidad real:KVM Switches


Exmaple5:

Texto de entrada:Stampcolo

In [59]:
# Ejemplo de uso
k = 50
for index in range(10):
    print(f'Exmaple{index}:\n')
    nuevo_texto = test['title'].iloc[index]
    entidad_clasificada = clasificar_texto(nuevo_texto, k)
    print(f"Texto de entrada:{nuevo_texto}")
    print(f"Entidad clasificada: {entidad_clasificada}")
    print(f"Entidad real:{test['category_name'].iloc[index]}")
    print('\n')

Exmaple0:

Texto de entrada:Arrow Shed Classic 8' x 6' Outdoor Padlockable Steel Storage Shed Building, Sage Green
Entidad clasificada: Outdoor Storage  Housing
Entidad real:Outdoor Storage  Housing


Exmaple1:

Texto de entrada:Berghaus Women's 24/7 Half Zip Long Sleeve Tech T-Shirt
Entidad clasificada: Sports & Outdoors
Entidad real:Sports & Outdoors


Exmaple2:

Texto de entrada:SilverAlloy 45% Ag BAg-5 18" Flux Coated Silver Solder Brazing Rods 3 Sticks
Entidad clasificada: Welding & Soldering
Entidad real:Welding & Soldering


Exmaple3:

Texto de entrada:Get Naked Bath Mat Cute Pink and White Bathroom Rugs Funny Non Slip Bathtub Decor Mats Super Absorbent Floor Carpet Machine Washable Bahtmat for Tub, Shower, Bedroom 16"x24"
Entidad clasificada: Bath Products
Entidad real:Home Décor Products


Exmaple4:

Texto de entrada:HP JC900A Tippingpoint 80mm Fan Module - Other Accessories
Entidad clasificada: Computer Components
Entidad real:KVM Switches


Exmaple5:

Texto de entrada:Stampc